In [ ]:
# Load data from a folder

from horsefm.datasets import clipper_datasets

clipper_repo = '/archives/celestia/Master File'
dialogue_dataset = clipper_datasets.mlp_dialogue_dataset(clipper_repo)
data = dialogue_dataset.pandas()

print('done')

In [ ]:
# Convert to a HuggingFace dataset

from datasets import Dataset, Audio

transcript = data['transcript']
audio_path = data['audio_path']
end = data['end']
datapoints = (data[audio_path.notna() & end.notna()]
    .rename(columns={
        'audio_path': 'audio',
        'character': 'speaker',
        'tags': 'style',
        'transcript': 'transcription',
    })
    
)
datapoints['style'] = datapoints['style'].apply(lambda x: ','.join(x))
datapoints['duration'] = datapoints['end'] - datapoints['start']

dataset = Dataset.from_dict(datapoints.to_dict(orient='list')).cast_column("audio", Audio())

In [ ]:
# Push to HuggingFace Hub

dataset.push_to_hub("synthbot/pony-speech")